# Process GREIN Rat Data

Retrieve the downloaded expression data, update gene identifiers to entrez, and curate sample IDs. The script will also identify a balanced hold-out test set to compare projection performance into learned latent spaces across algorithms.

In [27]:
import os
import random
import pandas as pd
from sklearn.model_selection import train_test_split

In [28]:
random.seed(1234)

## Read Phenotype Information: skipping until phenotype data found

In [29]:
# path = os.path.join('download', 'TARGET_phenotype.gz')
# pheno_df = pd.read_table(path)

# print(pheno_df.shape)
# pheno_df.head(3)

## Read Probe Mapping Info 
(where chromosomes start and end, saved in a file in downloads)

In [30]:
# path = os.path.join('download', 'gencode.v23.annotation.gene.probemap')
# probe_map_df = pd.read_table(path)

# # Inner merge gene df to get ensembl to entrez mapping
# probe_map_df = probe_map_df.merge(gene_df, how='inner', left_on='gene', right_on='symbol')

# # Mapping to rename gene expression index
# ensembl_to_entrez = dict(zip(probe_map_df.id, probe_map_df.entrez_gene_id))

# print(probe_map_df.shape)
# probe_map_df.head(3)

## Read Gene Expression Data

In [31]:
# reading in rat gene expression data

file = os.path.join('download', 'grein_count_matrix_rat.pkl')
expr_df = pd.read_pickle(file)

print(expr_df.shape)
expr_df.head(4)

(18171, 39)


,gene,gene_symbol,GSM2668015,GSM2668016,GSM2668017,GSM2668018,GSM2668019,GSM2668020,GSM2668021,GSM2668022,GSM2674552,GSM2674553,GSM2674554,GSM2674555,GSM2674556,GSM2674557,GSM2674558,GSM2674559,GSM2674560,GSM2674561,GSM2674562,GSM2674563,GSM2674564,GSM2674565,GSM2674566,GSM2674567,GSM2683306,GSM2683307,GSM2683308,GSM2683309,GSM2683310,GSM2683311,GSM2683312,GSM2699989,GSM2699990,GSM2699991,GSM2699992,GSM2699993,GSM2699994
0,ENSRNOG00000000007,Gad1,5140.2551,2873.5173,3494.3567,2488.5614,2180.0086,4054.4603,3249.2422,2984.1460,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,52.5118,27.5834,70.6123,55.3001,60.6636,64.8627,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
1,ENSRNOG00000000008,Alx4,88.3117,54.3955,68.8133,54.5058,61.3839,84.8890,92.7423,62.1657,0.0,0.0,0.0,2.0,0.0,2.0,1.2676,1.0,0.0,1.0,0.0,2.0,1.0000,0.0,0.0,2.0,101.9639,77.8544,77.2475,67.0283,61.0076,92.9832,75.9176,10.0072,10.2904,12.9965,5.2917,2.9983,2.9982
2,ENSRNOG00000000009,Tmco5b,71.9091,47.7813,50.4332,41.1147,40.6470,56.4535,48.4759,47.5353,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.9999,0.0,0.0,0.0,76.9727,42.2429,54.0743,83.0249,56.5682,75.6148,44.3473,0.0000,0.0000,1.9925,0.0000,0.0000,0.0000
3,ENSRNOG00000000010,Cbln1,366.8392,236.4766,388.1470,212.8647,218.2449,392.4068,325.6375,231.4440,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,126.6326,97.5128,89.0460,141.5510,100.1295,143.5802,87.1007,80.7889,95.8700,88.7592,0.0000,0.9978,0.0000


In [32]:
# Clean up x and y from expr_df

# create a list of cleaned columns
cleaned_column = ['gene', 'gene_symbol']
for column in expr_df.drop(['gene', 'gene_symbol'], axis = 1).columns:
    cleaned_column.append(column.split("_")[0])

# set colums to cleaned column
expr_df.columns = cleaned_column

In [33]:
# Check that it worked

pd.set_option('display.max_columns', None) 
expr_df.head()

,gene,gene_symbol,GSM2668015,GSM2668016,GSM2668017,GSM2668018,GSM2668019,GSM2668020,GSM2668021,GSM2668022,GSM2674552,GSM2674553,GSM2674554,GSM2674555,GSM2674556,GSM2674557,GSM2674558,GSM2674559,GSM2674560,GSM2674561,GSM2674562,GSM2674563,GSM2674564,GSM2674565,GSM2674566,GSM2674567,GSM2683306,GSM2683307,GSM2683308,GSM2683309,GSM2683310,GSM2683311,GSM2683312,GSM2699989,GSM2699990,GSM2699991,GSM2699992,GSM2699993,GSM2699994
0,ENSRNOG00000000007,Gad1,5140.2551,2873.5173,3494.3567,2488.5614,2180.0086,4054.4603,3249.2422,2984.1460,0.0,0.0000,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0,0.0,52.5118,27.5834,70.6123,55.3001,60.6636,64.8627,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
1,ENSRNOG00000000008,Alx4,88.3117,54.3955,68.8133,54.5058,61.3839,84.8890,92.7423,62.1657,0.0,0.0000,0.0,2.0000,0.0,2.0,1.2676,1.0,0.0,1.0,0.0,2.0,1.0000,0.0000,0.0,2.0,101.9639,77.8544,77.2475,67.0283,61.0076,92.9832,75.9176,10.0072,10.2904,12.9965,5.2917,2.9983,2.9982
2,ENSRNOG00000000009,Tmco5b,71.9091,47.7813,50.4332,41.1147,40.6470,56.4535,48.4759,47.5353,0.0,0.0000,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.9999,0.0000,0.0,0.0,76.9727,42.2429,54.0743,83.0249,56.5682,75.6148,44.3473,0.0000,0.0000,1.9925,0.0000,0.0000,0.0000
3,ENSRNOG00000000010,Cbln1,366.8392,236.4766,388.1470,212.8647,218.2449,392.4068,325.6375,231.4440,0.0,0.0000,0.0,0.0000,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0,0.0,126.6326,97.5128,89.0460,141.5510,100.1295,143.5802,87.1007,80.7889,95.8700,88.7592,0.0000,0.9978,0.0000
4,ENSRNOG00000000012,Tcf15,164.0565,86.1982,86.2113,86.0566,100.7740,145.9362,98.5538,97.6332,0.0,1.7998,0.0,1.4823,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0000,2.4423,0.0,0.0,31.8007,33.2070,33.8593,47.3560,26.0843,30.6765,43.2454,9.9237,13.7869,6.8718,3.3902,5.8108,10.7463


## Process gene expression matrix

This involves updating Entrez gene ids, sorting and subsetting

In [34]:
# expr_df = (expr_df
#     .dropna(axis='rows')
#     .reindex(probe_map_df.id)
#     .rename(index=ensembl_to_entrez)
#     .rename(index=old_to_new_entrez)
#     .groupby(level=0).mean()
#     .transpose()
#     .sort_index(axis='rows')
#     .sort_index(axis='columns')
# )

expr_df.index.rename('sample_id', inplace=True)

print(expr_df.shape)
expr_df.head(2)

(18171, 39)


,gene,gene_symbol,GSM2668015,GSM2668016,GSM2668017,GSM2668018,GSM2668019,GSM2668020,GSM2668021,GSM2668022,GSM2674552,GSM2674553,GSM2674554,GSM2674555,GSM2674556,GSM2674557,GSM2674558,GSM2674559,GSM2674560,GSM2674561,GSM2674562,GSM2674563,GSM2674564,GSM2674565,GSM2674566,GSM2674567,GSM2683306,GSM2683307,GSM2683308,GSM2683309,GSM2683310,GSM2683311,GSM2683312,GSM2699989,GSM2699990,GSM2699991,GSM2699992,GSM2699993,GSM2699994
sample_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,ENSRNOG00000000007,Gad1,5140.2551,2873.5173,3494.3567,2488.5614,2180.0086,4054.4603,3249.2422,2984.1460,0.0,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,52.5118,27.5834,70.6123,55.3001,60.6636,64.8627,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
1,ENSRNOG00000000008,Alx4,88.3117,54.3955,68.8133,54.5058,61.3839,84.8890,92.7423,62.1657,0.0,0.0,0.0,2.0,0.0,2.0,1.2676,1.0,0.0,1.0,0.0,2.0,1.0,0.0,0.0,2.0,101.9639,77.8544,77.2475,67.0283,61.0076,92.9832,75.9176,10.0072,10.2904,12.9965,5.2917,2.9983,2.9982


## Stratify Balanced Training and Testing Sets in TARGET Gene Expression

Output training and testing gene expression datasets

In [35]:
#strat = pheno_df.set_index('sample_id').reindex(expr_df.index).primary_disease_code

In [36]:
# cancertype_count_df = (
#     pd.DataFrame(strat.value_counts()) #not using value_counts, what number should we use here? 
#     .reset_index()
#     .rename({'index': 'cancertype', 'primary_disease_code': 'n ='}, axis='columns')
# )

# file = os.path.join('data', 'target_sample_counts.tsv') #change which file - do we have a file that works for this?
# cancertype_count_df.to_csv(file, sep='\t', index=False)

# cancertype_count_df

In [37]:
train_df, test_df = train_test_split(expr_df,
                                     test_size=0.1,
                                     random_state=123) #if no stratify defined, should just randomize on its own

In [38]:
print(train_df.shape)
test_df.shape

(16353, 39)


(1818, 39)

In [39]:
#save train dataframe to file 
train_file = os.path.join('data', 'train_grein_rat_expression_matrix_processed.tsv.gz')
train_df.to_csv(train_file, sep='\t', compression='gzip', float_format='%.3g')

In [40]:
#save test dataframe to file 
test_file = os.path.join('data', 'test_grein_rat_expression_matrix_processed.tsv.gz')
test_df.to_csv(test_file, sep='\t', compression='gzip', float_format='%.3g')

## Sort genes based on median absolute deviation and output to file

In [41]:
# function to calculate median absolute deviation
def mad(df): 
    """Function to calculate median absolute deviation for a dataframe
  argument1 (dataframe): Dataframe for which to calculate the median absolute deviation, row by row

  Returns: The median absolute deviation for each row of the data frame
  """
    row_medians = df.median(axis='columns', numeric_only=True)
    abs_row_median_diffs = abs(df.sub(row_medians, axis='rows'))
    return abs_row_median_diffs.median(axis='columns', numeric_only=True)

In [42]:
# add gene_id as a column in dataframe
train_df['gene_id'] = train_df['gene'] + train_df['gene_symbol']
col = train_df.pop('gene_id')
train_df.insert(0, col.name, col)

train_df.head()

,gene_id,gene,gene_symbol,GSM2668015,GSM2668016,GSM2668017,GSM2668018,GSM2668019,GSM2668020,GSM2668021,GSM2668022,GSM2674552,GSM2674553,GSM2674554,GSM2674555,GSM2674556,GSM2674557,GSM2674558,GSM2674559,GSM2674560,GSM2674561,GSM2674562,GSM2674563,GSM2674564,GSM2674565,GSM2674566,GSM2674567,GSM2683306,GSM2683307,GSM2683308,GSM2683309,GSM2683310,GSM2683311,GSM2683312,GSM2699989,GSM2699990,GSM2699991,GSM2699992,GSM2699993,GSM2699994
sample_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3584,ENSRNOG00000007336Churc1,ENSRNOG00000007336,Churc1,322.7576,237.0001,250.0003,186.1641,221.0005,317.0003,250.6698,235.0003,126.0941,235.1763,355.2647,479.3631,488.3686,465.3487,473.3577,378.5225,187.7590,185.1417,451.4255,441.3341,393.5051,326.2483,336.5778,414.4717,1801.3628,1435.1920,1473.0381,1613.7095,1278.1563,1707.6946,1893.1650,421.6607,433.8507,624.0168,656.0352,693.1579,555.9830
14684,ENSRNOG00000035610Mir377,ENSRNOG00000035610,Mir377,14.0000,7.0000,5.0000,8.0000,7.0000,6.0000,9.0000,3.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,2.9989,0.0000,0.0000,0.0000,2.0002,5.9990,2.0002,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
17130,ENSRNOG00000048445LOC100909862,ENSRNOG00000048445,LOC100909862,8.4862,11.3334,8.6440,9.1396,8.1458,14.6378,13.0723,6.8770,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,13.7399,4.4305,11.3453,0.0081,7.7333,7.0126,8.2752,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
66,ENSRNOG00000000169Spata5l1,ENSRNOG00000000169,Spata5l1,225.5967,150.5534,161.6999,129.2117,132.6296,188.6234,143.0324,132.2708,116.8599,102.3285,195.1699,179.9838,169.8034,161.4641,175.8895,186.4348,78.1931,88.6635,108.4148,124.4535,131.3203,98.6024,117.3997,129.5221,405.9823,348.4350,245.8922,376.0369,228.2067,371.6765,287.5426,153.5827,198.6694,239.0798,421.1322,405.5692,353.1912
17164,ENSRNOG00000048549Olr1240,ENSRNOG00000048549,Olr1240,31.3617,16.1621,14.9487,12.3349,11.9975,22.6926,24.3146,15.8916,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,19.7699,12.4066,13.0108,20.8329,10.6476,20.3194,18.4805,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000


In [43]:
# Determine most variably expressed genes and subset
# create dataframe to save median absolute deviation data for rats
train_df_mad = mad(train_df.drop(['gene_id','gene', 'gene_symbol'], axis = 1))
train_df_mad.columns = ['gene_id', 'median_abs_deviation']

train_df_mad = train_df_mad.sort_values(ascending=False)
train_df_mad

sample_id
14526    104386.5201
13990     98591.7197
13670     71535.2136
13750     68463.5045
13940     65865.1337
            ...     
17213         0.0000
16311         0.0000
17243         0.0000
8596          0.0000
9177          0.0000
Length: 16353, dtype: float64

In [44]:
# Save to tsv file
file = os.path.join('data', 'grein_mad_rat_genes.tsv')
train_df_mad.to_csv(file, sep='\t', index=False)